In [1]:
import pandas as pd
import numpy as np
import os
import rebound

path = '/scratch/dtamayo/'
icpath = path +'random/initial_conditions/runs/ic'
fcpath = path +'random/final_conditions/runs/fc'

df = pd.read_csv(path+'random/random.csv', index_col=0)

simIDmax = 25000
df = df.head(simIDmax)
df.tail()

,runstring
17995,0017995.bin
17996,0017996.bin
17997,0017997.bin
17998,0017998.bin
17999,0017999.bin


In [115]:
sim = rebound.Simulation.from_file(icpath+df.loc[11002].runstring)

/mnt/raid-cita/dtamayo/stability/lib/python3.4/site-packages/rebound/simulation.py:374: RuntimeWarning: You have to reset function pointers after creating a reb_simulation struct with a binary file.
  warnings.warn(message, RuntimeWarning)


In [105]:
columns = ['Stable','instability_time','Rel_Eerr','RHill12','RHill23','beta12', 'beta23']
for i in ['1', '2', '3']:
    columns += ['m'+i, 'a'+i, 'P'+i, 'e'+i, 'pomega'+i, 'inc'+i, 'Omega'+i, 'f'+i]

In [106]:
def get_initial_orbital_elements(row):
    try:
        sim = rebound.Simulation.from_file(icpath+row["runstring"])
        simf = rebound.Simulation.from_file(fcpath+row["runstring"])
        ps = sim.particles
        stable = 1 if np.isclose(simf.t, 1.e9) else 0
        instability_time = simf.t
        Rel_Eerr = abs((simf.calculate_energy()-sim.calculate_energy())/sim.calculate_energy())
        RHill12 = ps[1].a*((ps[1].m + ps[2].m)/(3.*ps[0].m))**(1./3.)
        RHill23 = ps[2].a*((ps[2].m + ps[3].m)/(3.*ps[0].m))**(1./3.)
        beta12 = (ps[2].a - ps[1].a)/RHill12
        beta23 = (ps[3].a - ps[2].a)/RHill23
        features = [stable,instability_time,Rel_Eerr,RHill12,RHill23,beta12,beta23]
        for i in [1,2,3]:
            features += [ps[i].m, ps[i].a, ps[i].P, ps[i].e, ps[i].pomega, ps[i].inc, ps[i].Omega, ps[i].f]
        return pd.Series(features, index=columns)    
    except:
        return np.nan

In [107]:
%%time
df = pd.concat([df, df.apply(get_initial_orbital_elements, axis=1)], axis=1)

/mnt/raid-cita/dtamayo/stability/lib/python3.4/site-packages/rebound/simulation.py:374: RuntimeWarning: You have to reset function pointers after creating a reb_simulation struct with a binary file.
  warnings.warn(message, RuntimeWarning)


CPU times: user 15.7 s, sys: 359 ms, total: 16.1 s
Wall time: 16.5 s


In [108]:
df = df.drop([11003])
df['Stable'] = df['Stable'].astype(int)

In [109]:
df.to_csv('../csvs/initial_orbital_elements.csv', encoding='ascii')